In [1]:
import pandas as pd
import numpy as np

store_data = pd.read_csv("./data/store.csv")
store_data

train_data = pd.read_csv("./data/train.csv")
train_data

/Users/corstiaen/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0
...,...,...,...,...,...,...,...,...,...
637769,2014-07-31,752.0,4.0,7259.0,576.0,1.0,1.0,0,1.0
637770,2014-07-31,753.0,NaN,9938.0,796.0,1.0,1.0,0,1.0
637771,2014-07-31,754.0,4.0,10564.0,1162.0,1.0,1.0,0,NaN
637772,2014-07-31,755.0,4.0,12302.0,1020.0,1.0,1.0,0,1.0


In [2]:
train_data.drop("Customers", axis=1, inplace=True)

In [4]:
from sklearn.model_selection import train_test_split

X = train_data.loc[:, ~train_data.columns.isin(["Sales"])]
y = train_data.loc[:, "Sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=.8)

In [12]:
X_train.head()

,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
143027,2013-05-09,83.0,4.0,0.0,NaN,a,0.0
553420,2014-05-12,68.0,1.0,NaN,0.0,0,0.0
589010,2014-06-13,88.0,5.0,1.0,0.0,0,0.0
227227,2013-07-23,685.0,2.0,1.0,0.0,0,1.0
561723,2014-05-19,689.0,1.0,1.0,1.0,0,0.0


In [13]:
len(X_train)

510219

In [14]:
len(X_test)

127555

In [7]:
from pandas_profiling import ProfileReport

profile = ProfileReport(X_train)
profile.to_file(output_file="rossman_profile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/corstiaen/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]